# Ridge Classifier

Ridge Classifier is a type of linear classifier that uses L2 regularization to prevent overfitting. It is particularly useful when the data has multicollinearity, as it adds a penalty to the coefficients, shrinking them towards zero. This helps in improving the generalization of the model. You can check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html#sklearn.linear_model.RidgeClassifier) for more details.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports and path setup
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
from sklearn.utils import shuffle
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from userkits.features import *
from userkits.utils import load_train_data, load_eval_data

## Load and Shuffle Data

In [3]:
# load data from train and eval directories
X, y = load_train_data(data_dir='../train')
X, y = shuffle(X, y, random_state=42)
X_eval = load_eval_data(data_dir='../eval')

## Transform Data and Add Features

The steps to include new features are detailed in (the file). You can find the definitions of currently included features there.

In [4]:
def extract_features(images):
    features_list = []
    def process_image(img):
        feats = []
        # add feature functions here
        feats.extend(color_histogram(img))
        feats.extend(lbp_texture_features(img))
        feats.extend(find_mean(img))
        feats.extend(find_stddev(img))
        return feats

    features_list = Parallel(n_jobs=-1)(delayed(process_image)(img) for img in tqdm(images, desc="Extracting features"))
    return np.array(features_list)

In [5]:
X_features = extract_features(X)
X_features.shape

Extracting features:   0%|          | 0/17735 [00:00<?, ?it/s]

(17735, 528)

In [6]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

## Common Hyperparameters
- `alpha`: Controls the strength of the L2 penalty. A higher value increases regularization, reducing the model's complexity, while a lower value allows the model to fit the data more closely.  
- `fit_intercept`: determines whether to calculate the intercept for this model. If set to `False`, no intercept will be used in calculations.

In [9]:
# Split the data and train the model
X_train, X_test, y_train, y_test = train_test_split(X_features, y_encoded, test_size=0.2)  # you can change test_size
clf = RidgeClassifier()  # you can tune hyperparameters here
clf.fit(X_train, y_train)
print("Train Accuracy:", clf.score(X_train, y_train))
print("Test Accuracy:", clf.score(X_test, y_test))

Train Accuracy: 0.4849873132224415
Test Accuracy: 0.4383986467437271


## Evaluate

In [8]:
X_eval_features = extract_features(X_eval)
eval_predictions = clf.predict(X_eval_features)
print(eval_predictions[:5])

Extracting features:   0%|          | 0/4434 [00:00<?, ?it/s]

[31 31 36 18  3]


## Submit